In [ ]:
import win32com.client
from win32com.client import Dispatch
import datetime as date
import os.path
import csv, openpyxl
from openpyxl import load_workbook
import xlsxwriter
import pandas as pd
from pandas import DataFrame
import pyautogui as pya
import time

In [ ]:
def analyze_ER_Hours(number):
    subject = "FW: Worklog Reports for Enhancement Requests" # email subject line that gets auto-forwarded to me
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.GetDefaultFolder(6) # default Inbox in Outlook

    report_name = "ER_" + str(number) + "_report.csv" #downloads worklog for specific JIRA ticket number in the email
    
    messages = inbox.Items
    messages.Sort("[ReceivedTime]", True) # sorts the inbox from most recent to oldest
    todays_date = date.date.today()
    sub_today = subject
    att_today = report_name

    for msg in messages:
        if msg.Subject == sub_today:
             break
    for att in msg.Attachments:
        if att.FileName == att_today:
             break
    att.SaveASFile(os.getcwd() + '\\' + att.FileName)
    print("Mail Successfully Extracted") # confirms successful download
    

    ER_hours_analysis_wb = "ER Hours Analysis " + str(number) + ".xlsx" # new workbook for analysis

    df = pd.read_csv("ER_"+str(number)+"_Report.csv") # reading the .csv to a DataFrame
    df2 = df
    for i in range(len(df["Start Date"])):
        d=df["Start Date"].values[i]
        df2["Start Date"].values[i] = date.datetime.strptime(d, '%Y-%m-%d %H:%M:%S').strftime('%m/%#d/%#Y') # stripping the time

    df2.set_index("Jira Item", inplace=True)
    df2.to_csv("ER_"+str(number)+"_Report.csv", index=False)
    
    book = load_workbook(ER_hours_analysis_wb, read_only=False, data_only=False)
  

    sheet_name = "ER Work Log"
    with pd.ExcelWriter("Hours_Analysis_" + str(number) + ".xlsx", engine='openpyxl')  as writer:

        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

        df2.to_excel(writer, sheet_name=sheet_name, startrow=1, startcol=0, engine='openpyxl')
        
        wb = load_workbook(ER_hours_analysis_wb)
        ws = wb["ER Work Log"]  
       
    file = "Hours_Analysis_"+str(number)+".xlsx"
    f = os.startfile(file)  
    time.sleep(10)
    #pya.FAILSAFE = False
    #pya.PAUSE = 10 # give Excel 10 sec to open 
    #pya.press('down')
    pya.hotkey('ctrl', 'pagedown')
    #pya.PAUSE = 0.2
    pya.hotkey('ctrl', 'shift', 'down')
    pya.hotkey('alt', 'a', 'e')
    pya.press(['tab', 'tab', 'tab'])
    pya.press('enter')
    pya.hotkey('alt', 'h', 'n')
    pya.press(['s','h','enter'])
    pya.hotkey('ctrl', 's')      
    # the time stamp for work was not formatting properly when pasted into Excel so I 
    # added this super kludgey set of hot keys commands to properly format the time :/